### <left> High-Dimensional Deep Learning - 5A
## <center>*Mini-project n° 3 – SSL for Anomaly Detection*</center>
### <center>2024-2025</center>
<div style="text-align: right;">
    <b>Project 3/4</b><br>
<div style="text-align: left;">
<b>Noms :</b> Maïmouna Gadji, Phuc-Luan Nguyen, Maddie Perez et Julia Soufflet<br>

### Objectif : 

L'objectif de ce projet est de 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import shutil
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
# Training images
pathb="./MVTec_AD/bottle"
pathh = "./MVTec_AD/hazelnut"
pathc = "./MVTec_AD/capsule"
patht = "./MVTec_AD/toothbrush"
pathw = "./engine_wiring"

In [3]:
# Training images
train_filenames_bottle = os.listdir(pathb + "/train/good")
train_filenames_hazelnut = os.listdir(pathh + "/train/good")
train_filenames_capsule = os.listdir(pathc + "/train/good")
train_filenames_toothbrush = os.listdir(patht + "/train/good")
train_filenames_W = os.listdir(pathw + "/train/good")
#Création des labels
train_categories_B = []
for filename in train_filenames_bottle:
    train_categories_B.append(0)

train_categories_H = []
for filename in train_filenames_hazelnut:
    train_categories_H.append(0)

train_categories_C = []
for filename in train_filenames_capsule:
    train_categories_C.append(0)

train_categories_T = []
for filename in train_filenames_toothbrush:
    train_categories_T.append(0)

train_categories_W = []
for filename in train_filenames_W:
    train_categories_W.append(0)

In [4]:

def reorganize_images_good(source_dirs, target_dir):
    """
    Cette fonction regroupe les images des différents dossiers (good et défauts) et les renomme avec des noms uniques.
    
    :param source_dirs: Liste des chemins vers les dossiers à traiter (good et les dossiers défauts)
    :param target_dir: Dossier où toutes les images seront regroupées et renommées
    """
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)  # Crée le dossier cible s'il n'existe pas
    
    count = 1  # Compteur pour nommer les fichiers de manière unique

    # Parcours chaque dossier dans la liste source_dirs
    for source_dir in source_dirs:
        for filename in os.listdir(source_dir):
            file_path = os.path.join(source_dir, filename)
            
            # Vérifie si c'est un fichier image (vous pouvez étendre les types de fichiers si nécessaire)
            if os.path.isfile(file_path) and filename.endswith(('.png')):
                # Nouveau nom basé sur le compteur
                new_name = f"good{str(count).zfill(3)}.png"  # 001.jpg, 002.jpg, etc.
                target_path = os.path.join(target_dir, new_name)
                
                # Copie le fichier vers le dossier cible avec le nouveau nom
                shutil.copy(file_path, target_path)
                
                # Incrémente le compteur pour le prochain fichier
                count += 1

def reorganize_images_default(source_dirs, target_dir):
    """
    Cette fonction regroupe les images des différents dossiers (good et défauts) et les renomme avec des noms uniques.
    
    :param source_dirs: Liste des chemins vers les dossiers à traiter (good et les dossiers défauts)
    :param target_dir: Dossier où toutes les images seront regroupées et renommées
    """
    if not os.path.exists(target_dir):
        os.makedirs(target_dir)  # Crée le dossier cible s'il n'existe pas
    
    count = 1  # Compteur pour nommer les fichiers de manière unique

    # Parcours chaque dossier dans la liste source_dirs
    for source_dir in source_dirs:
        for filename in os.listdir(source_dir):
            file_path = os.path.join(source_dir, filename)
            
            # Vérifie si c'est un fichier image (vous pouvez étendre les types de fichiers si nécessaire)
            if os.path.isfile(file_path) and filename.endswith(('.png')):
                # Nouveau nom basé sur le compteur
                new_name = f"default{str(count).zfill(3)}.png"  # 001.jpg, 002.jpg, etc.
                target_path = os.path.join(target_dir, new_name)
                
                # Copie le fichier vers le dossier cible avec le nouveau nom
                shutil.copy(file_path, target_path)
                
                # Incrémente le compteur pour le prochain fichier
                count += 1

In [5]:
# Test bottle
source_dirs_D = [
    pathb+'/test/contamination',
    pathb+'/test/broken_small',
    pathb+'/test/broken_large'# Dossier avec des défauts (MVT-AD)
 
]

source_dirs_G = [
    pathb+'/test/good',  # Dossier 'good'
 ]

target_dirB = pathb +'/test/test/'  # Dossier cible

# Appeler la fonction pour organiser les images
reorganize_images_good(source_dirs_G, target_dirB)
reorganize_images_default(source_dirs_D, target_dirB)

#Création des labels
test_filenames_B = os.listdir(target_dirB)
test_categories_B = []
for filename in test_filenames_B:
    category = filename.split('.')[0][:7]
    if category == 'default':
        test_categories_B.append(1)
    else:
        test_categories_B.append(0)

In [6]:
# Test hazelnut
source_dirs_D = [
    pathh+'/test/crack',
    pathh+'/test/cut',
    pathh+'/test/print',
    pathh+'/test/hole'# Dossier avec des défauts (MVT-AD)
 
]

source_dirs_G = [
    pathh+'/test/good',  # Dossier 'good'
 ]

target_dirH = pathh +'/test/test'  # Dossier cible

# Appeler la fonction pour organiser les images
reorganize_images_good(source_dirs_G, target_dirH)
reorganize_images_default(source_dirs_D, target_dirH)

#Création des labels
test_filenames_H = os.listdir(target_dirH)
test_categories_H = []
for filename in test_filenames_H:
    category = filename.split('.')[0][:7]
    if category == 'default':
        test_categories_H.append(1)
    else:
        test_categories_H.append(0)

In [7]:
# Test capsule
source_dirs_D = [
    pathc+'/test/crack',
    pathc+'/test/faulty_imprint',
    pathc+'/test/poke',
    pathc+'/test/scratch',
    pathc+'/test/squeeze'# Dossier avec des défauts (MVT-AD)
 
]

source_dirs_G = [
    pathc+'/test/good',  # Dossier 'good'
 ]

target_dirC = pathc +'/test/test'  # Dossier cible

# Appeler la fonction pour organiser les images
reorganize_images_good(source_dirs_G, target_dirC)
reorganize_images_default(source_dirs_D, target_dirC)

#Création des labels
test_filenames_C = os.listdir(target_dirC)
test_categories_C = []
for filename in test_filenames_C:
    category = filename.split('.')[0][:7]
    if category == 'default':
        test_categories_C.append(1)
    else:
        test_categories_C.append(0)

In [8]:
# Test toothbrush
source_dirs_D = [
    patht+'/test/defective'# Dossier avec des défauts (MVT-AD)
 
]

source_dirs_G = [
    patht+'/test/good',  # Dossier 'good'
 ]

target_dirT = patht +'/test/test'  # Dossier cible

# Appeler la fonction pour organiser les images
reorganize_images_good(source_dirs_G, target_dirT)
reorganize_images_default(source_dirs_D, target_dirT)

#Création des labels
test_filenames_T = os.listdir(target_dirT)
test_categories_T = []
for filename in test_filenames_T:
    category = filename.split('.')[0][:7]
    if category == 'default':
        test_categories_T.append(1)
    else:
        test_categories_T.append(0)

In [9]:
# Test engine wiring
source_dirs_D = [
    pathw+'/test/blue_hoop',
    pathw+'/test/cardboard',
    pathw+'/test/fastening',
    pathw+'/test/multiple',
    pathw+'/test/obstruction'# Dossier avec des défauts (Auto_VI)
 
]

source_dirs_G = [
    pathw+'/test/good',  # Dossier 'good'
 ]

target_dirW = pathw +'/test/test'  # Dossier cible

# Appeler la fonction pour organiser les images
reorganize_images_good(source_dirs_G, target_dirW)
reorganize_images_default(source_dirs_D, target_dirW)

#Création des labels
test_filenames_W = os.listdir(target_dirW)
test_categories_W = []
for filename in test_filenames_W:
    category = filename.split('.')[0][:7]
    if category == 'default':
        test_categories_W.append(1)
    else:
        test_categories_W.append(0)

PermissionError: [Errno 13] Permission denied: './engine_wiring/test/test/good001.png'

In [10]:
# Training images
total_train_df_bottle = pd.DataFrame({
    'filename': train_filenames_bottle,
    'category': train_categories_B
})
# Test images
test_df_bottle = pd.DataFrame({
    'filename': test_filenames_B,
    'category': test_categories_B
})
# Mise à jour des chemins dans le DataFrame pour inclure les extensions correctes
total_train_df_bottle['filename'] = total_train_df_bottle['filename'].apply(lambda x: os.path.join(pathb+'/train/good', x))
test_df_bottle['filename'] = test_df_bottle['filename'].apply(lambda x: os.path.join(target_dirB, x))
print(total_train_df_bottle)

                                 filename  category
0    ./MVTec_AD/bottle/train/good/000.png         0
1    ./MVTec_AD/bottle/train/good/001.png         0
2    ./MVTec_AD/bottle/train/good/002.png         0
3    ./MVTec_AD/bottle/train/good/003.png         0
4    ./MVTec_AD/bottle/train/good/004.png         0
..                                    ...       ...
204  ./MVTec_AD/bottle/train/good/204.png         0
205  ./MVTec_AD/bottle/train/good/205.png         0
206  ./MVTec_AD/bottle/train/good/206.png         0
207  ./MVTec_AD/bottle/train/good/207.png         0
208  ./MVTec_AD/bottle/train/good/208.png         0

[209 rows x 2 columns]


In [11]:
# Training images
total_train_df_hazelnut = pd.DataFrame({
    'filename': train_filenames_hazelnut,
    'category': train_categories_H
})

# Test images
test_df_hazelnut = pd.DataFrame({
    'filename': test_filenames_H,
    'category': test_categories_H
})


# Mise à jour des chemins dans le DataFrame pour inclure les extensions correctes
total_train_df_hazelnut['filename'] = total_train_df_hazelnut['filename'].apply(lambda x: os.path.join(pathh+'/train/good', x))
test_df_hazelnut['filename'] = test_df_hazelnut['filename'].apply(lambda x: os.path.join(target_dirH, x))
print(total_train_df_hazelnut)

                                   filename  category
0    ./MVTec_AD/hazelnut/train/good/000.png         0
1    ./MVTec_AD/hazelnut/train/good/001.png         0
2    ./MVTec_AD/hazelnut/train/good/002.png         0
3    ./MVTec_AD/hazelnut/train/good/003.png         0
4    ./MVTec_AD/hazelnut/train/good/004.png         0
..                                      ...       ...
386  ./MVTec_AD/hazelnut/train/good/386.png         0
387  ./MVTec_AD/hazelnut/train/good/387.png         0
388  ./MVTec_AD/hazelnut/train/good/388.png         0
389  ./MVTec_AD/hazelnut/train/good/389.png         0
390  ./MVTec_AD/hazelnut/train/good/390.png         0

[391 rows x 2 columns]


In [12]:
# Training images
total_train_df_capsule = pd.DataFrame({
    'filename': train_filenames_capsule,
    'category': train_categories_C
})

# Test images
test_df_capsule = pd.DataFrame({
    'filename': test_filenames_C,
    'category': test_categories_C
})

# Mise à jour des chemins dans le DataFrame pour inclure les extensions correctes
total_train_df_capsule['filename'] = total_train_df_capsule['filename'].apply(lambda x: os.path.join(pathc+'/train/good', x))
test_df_capsule['filename'] = test_df_capsule['filename'].apply(lambda x: os.path.join(target_dirC, x))
print(total_train_df_capsule)

                                  filename  category
0    ./MVTec_AD/capsule/train/good/000.png         0
1    ./MVTec_AD/capsule/train/good/001.png         0
2    ./MVTec_AD/capsule/train/good/002.png         0
3    ./MVTec_AD/capsule/train/good/003.png         0
4    ./MVTec_AD/capsule/train/good/004.png         0
..                                     ...       ...
214  ./MVTec_AD/capsule/train/good/214.png         0
215  ./MVTec_AD/capsule/train/good/215.png         0
216  ./MVTec_AD/capsule/train/good/216.png         0
217  ./MVTec_AD/capsule/train/good/217.png         0
218  ./MVTec_AD/capsule/train/good/218.png         0

[219 rows x 2 columns]


In [13]:
# Training images
total_train_df_toothbrush = pd.DataFrame({
    'filename': train_filenames_toothbrush,
    'category': train_categories_T
})

# Test images
test_df_toothbrush = pd.DataFrame({
    'filename': test_filenames_T,
    'category': test_categories_T
})


# Mise à jour des chemins dans le DataFrame pour inclure les extensions correctes
total_train_df_toothbrush['filename'] = total_train_df_toothbrush['filename'].apply(lambda x: os.path.join(patht+'/train/good', x))
test_df_toothbrush['filename'] = test_df_toothbrush['filename'].apply(lambda x: os.path.join(target_dirT, x))
print(total_train_df_toothbrush)

                                    filename  category
0   ./MVTec_AD/toothbrush/train/good/000.png         0
1   ./MVTec_AD/toothbrush/train/good/001.png         0
2   ./MVTec_AD/toothbrush/train/good/002.png         0
3   ./MVTec_AD/toothbrush/train/good/003.png         0
4   ./MVTec_AD/toothbrush/train/good/004.png         0
5   ./MVTec_AD/toothbrush/train/good/005.png         0
6   ./MVTec_AD/toothbrush/train/good/006.png         0
7   ./MVTec_AD/toothbrush/train/good/007.png         0
8   ./MVTec_AD/toothbrush/train/good/008.png         0
9   ./MVTec_AD/toothbrush/train/good/009.png         0
10  ./MVTec_AD/toothbrush/train/good/010.png         0
11  ./MVTec_AD/toothbrush/train/good/011.png         0
12  ./MVTec_AD/toothbrush/train/good/012.png         0
13  ./MVTec_AD/toothbrush/train/good/013.png         0
14  ./MVTec_AD/toothbrush/train/good/014.png         0
15  ./MVTec_AD/toothbrush/train/good/015.png         0
16  ./MVTec_AD/toothbrush/train/good/016.png         0
17  ./MVTe

In [14]:
# Training images
total_train_df_W = pd.DataFrame({
    'filename': train_filenames_W,
    'category': train_categories_W
})

# Test images
test_df_W = pd.DataFrame({
    'filename': test_filenames_W,
    'category': test_categories_W
})


# Mise à jour des chemins dans le DataFrame pour inclure les extensions correctes
total_train_df_W['filename'] = total_train_df_W['filename'].apply(lambda x: os.path.join(pathw+'/train/good', x))
test_df_W['filename'] = test_df_W['filename'].apply(lambda x: os.path.join(target_dirW, x))
print(total_train_df_W)

NameError: name 'test_filenames_W' is not defined

# I - Partie 1 : Train SSL

In [42]:
class Encoder(nn.Module):
    def __init__(self, in_channels=3, latent_dim=128):
        super(Encoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=4, stride=2, padding=1),  # 32x32 -> 16x16
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),  # 16x16 -> 8x8
            nn.ReLU(),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),  # 8x8 -> 4x4
            nn.ReLU(),
            nn.Conv2d(256, latent_dim, kernel_size=4, stride=2, padding=1),  # 4x4 -> 2x2
            nn.ReLU(),
        )

    def forward(self, x):
        return self.encoder(x)

class Decoder(nn.Module):
    def __init__(self, latent_dim=128, out_channels=3):
        super(Decoder, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 256, kernel_size=4, stride=2, padding=1),  # 2x2 -> 4x4
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),  # 4x4 -> 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),  # 8x8 -> 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(64, out_channels, kernel_size=4, stride=2, padding=1),  # 16x16 -> 32x32
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.decoder(x)

In [46]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import pandas as pd

# Définir un Dataset personnalisé
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        """
        Args:
            dataframe (pd.DataFrame): DataFrame contenant les colonnes ['path', 'label'].
            transform (callable, optional): Transformations à appliquer sur les images.
        """
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx, 0]
        label = self.dataframe.iloc[idx, 1]

        try:
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
        except Exception as e:
            print(f"Erreur lors du chargement de l'image {img_path}: {e}")
            return None

        return image, torch.tensor(label, dtype=torch.long)

# Exemple d'utilisation
def create_dataloader(dataframe, batch_size=128, image_size=(32, 32), shuffle=True):
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
    ])
    dataset = CustomDataset(dataframe, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=2)
    return dataloader

# Exemple de DataFrame

# Création des DataLoaders pour "bottle"
dataloader_train_b = create_dataloader(total_train_df_bottle, batch_size=1)
dataloader_test_b = create_dataloader(test_df_bottle, batch_size=1)

# Création des DataLoaders pour "hazelnut"
dataloader_train_h = create_dataloader(total_train_df_hazelnut, batch_size=1)
dataloader_test_h = create_dataloader(test_df_hazelnut, batch_size=1)

# Création des DataLoaders pour "capsule"
dataloader_train_c = create_dataloader(total_train_df_capsule, batch_size=1)
dataloader_test_c = create_dataloader(test_df_capsule, batch_size=1)

# Création des DataLoaders pour "toothbrush"
dataloader_train_t = create_dataloader(total_train_df_toothbrush, batch_size=1)
dataloader_test_t = create_dataloader(test_df_toothbrush, batch_size=1)

# Création des DataLoaders pour EW (w)
#dataloader_train_w = create_dataloader(total_train_df_W, batch_size=1)
#dataloader_test_w = create_dataloader(test_df_W, batch_size=1)


In [47]:
def train_ssl_model(model, 
                    train_loader, 
                    test_loader, 
                    criterion,
                    optimizer,
                    device=device,
                    epochs=5):

    for epoch in range(epochs):
        model.to(device)
        model.train()
        total_train_loss = 0
        for images, _ in train_loader:
            images = images.to(device)
            optimizer.zero_grad()
            output, _ = model(images)
            loss = criterion(output, images)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)

        model.eval()
        total_val_loss = 0
        with torch.no_grad():
            for images, _ in test_loader:
                images = images.to(device)
                output, _ = model(images)
                val_loss = criterion(output, images)
                total_val_loss += val_loss.item()

        avg_val_loss = total_val_loss / len(test_loader)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Avg Val Loss: {avg_val_loss:.4f}")
    
    return model.encoder

## Pretext tasks 1 : colorizing model

Explications.

In [48]:
class ColorizationModel(nn.Module):
    def __init__(self, latent_dim=128):
        super(ColorizationModel, self).__init__()
        self.encoder = Encoder(latent_dim=latent_dim, in_channels=1)  # Input grayscale
        self.decoder = Decoder(latent_dim=latent_dim, out_channels=3)  # Predict RGB

    def forward(self, x):
        grayscale_x = transforms.Grayscale(num_output_channels=1)(x)  # Convert RGB to Grayscale
        z = self.encoder(grayscale_x)
        return self.decoder(z), grayscale_x

### Sur MVTec_AD

In [49]:
print("Training bottle")
# Modèle de colorisation
colorization_model_bottle = ColorizationModel(latent_dim=128)  # Instantiate the colorization model

# Pré-entrainement pour "bottle"
colorization_model_bottle = train_ssl_model(
    colorization_model_bottle,
    dataloader_train_b,
    dataloader_test_b,
    criterion=nn.MSELoss(),
    optimizer=optim.Adam(colorization_model_bottle.parameters(), lr=0.001)
)

Training bottle
Epoch 1/5, Train Loss: 0.0080, Avg Val Loss: 0.0018
Epoch 2/5, Train Loss: 0.0009, Avg Val Loss: 0.0021
Epoch 3/5, Train Loss: 0.0009, Avg Val Loss: 0.0017
Epoch 4/5, Train Loss: 0.0008, Avg Val Loss: 0.0018
Epoch 5/5, Train Loss: 0.0008, Avg Val Loss: 0.0017


In [51]:
print("Training hazelnut")
colorization_model_hazelnut = ColorizationModel(latent_dim=128)  # Instantiate the colorization model

# Pré-entrainement pour "hazelnut"
colorization_encoder_h = train_ssl_model(
    colorization_model_hazelnut,
    dataloader_train_h,
    dataloader_test_h,
    criterion=nn.MSELoss(),
    optimizer=optim.Adam(colorization_model_hazelnut.parameters(), lr=0.001)
)

print("Training capsule")
colorization_model_capsule = ColorizationModel(latent_dim=128)  # Instantiate the colorization model

# Pré-entrainement pour "capsule"
colorization_encoder_c = train_ssl_model(
    colorization_model_capsule,
    dataloader_train_c,
    dataloader_test_c,
    criterion=nn.MSELoss(),
    optimizer=optim.Adam(colorization_model_capsule.parameters(), lr=0.001)
)

print("Training toothbrush")
colorization_model_toothbrush = ColorizationModel(latent_dim=128)  # Instantiate the colorization model

# Pré-entrainement pour "toothbrush"
colorization_encoder_t = train_ssl_model(
    colorization_model_toothbrush,
    dataloader_train_t,
    dataloader_test_t,
    criterion=nn.MSELoss(),
    optimizer=optim.Adam(colorization_model_toothbrush.parameters(), lr=0.001))

Training hazelnut
Epoch 1/5, Train Loss: 0.0053, Avg Val Loss: 0.0018
Epoch 2/5, Train Loss: 0.0018, Avg Val Loss: 0.0019
Epoch 3/5, Train Loss: 0.0018, Avg Val Loss: 0.0020
Epoch 4/5, Train Loss: 0.0016, Avg Val Loss: 0.0019
Epoch 5/5, Train Loss: 0.0012, Avg Val Loss: 0.0013
Training capsule
Epoch 1/5, Train Loss: 0.0049, Avg Val Loss: 0.0013
Epoch 2/5, Train Loss: 0.0010, Avg Val Loss: 0.0006
Epoch 3/5, Train Loss: 0.0007, Avg Val Loss: 0.0011
Epoch 4/5, Train Loss: 0.0007, Avg Val Loss: 0.0006
Epoch 5/5, Train Loss: 0.0006, Avg Val Loss: 0.0008
Training toothbrush
Epoch 1/5, Train Loss: 0.0210, Avg Val Loss: 0.0036
Epoch 2/5, Train Loss: 0.0026, Avg Val Loss: 0.0023
Epoch 3/5, Train Loss: 0.0020, Avg Val Loss: 0.0019
Epoch 4/5, Train Loss: 0.0017, Avg Val Loss: 0.0018
Epoch 5/5, Train Loss: 0.0016, Avg Val Loss: 0.0018


Interprétations.

In [50]:
import random

# Visualize colorization on random test images

def visualize_reconstructions(model, data_loader, device, num_images=5):
    # Set the model to evaluation mode
    model.eval()

    # Convert the DataLoader to a list to randomly sample images
    dataset = list(data_loader.dataset)

    # Randomly select `num_images` images from the dataset
    random_indices = random.sample(range(len(dataset)), num_images)
    random_images = [dataset[i][0] for i in random_indices]  # Extract only the images, ignoring labels

    # Stack the images into a batch
    images = torch.stack(random_images)

    # Move images to the specified device
    images = images.to(device)
    
    # Run the grayscale images through the colorization model
    with torch.no_grad():
        reconstructed_images, perturbed_images = model(images)
    
    # Move images back to CPU for visualization
    images = images.cpu()
    reconstructed_images = reconstructed_images.cpu()
    perturbed_images = perturbed_images.cpu()
    
    # Plot the grayscale, ground truth, and colorized images
    fig, axes = plt.subplots(num_images, 3, figsize=(10, num_images * 4))
    for i in range(num_images):
        # Grayscale input
        axes[i, 0].imshow(perturbed_images[i].permute(1, 2, 0).squeeze(), cmap='gray')
        axes[i, 0].set_title("Grayscale Input")
        axes[i, 0].axis('off')
        
        # Ground truth (original RGB image)
        axes[i, 1].imshow(images[i].permute(1, 2, 0))
        axes[i, 1].set_title("Ground Truth (RGB)")
        axes[i, 1].axis('off')
        
        # Colorized output from the model
        axes[i, 2].imshow(reconstructed_images[i].permute(1, 2, 0))
        axes[i, 2].set_title("Colorized Output")
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.show()

# Visualize colorization on random test images
visualize_reconstructions(colorization_model_bottle, dataloader_test_b, device)

RuntimeError: Given groups=1, weight of size [64, 1, 4, 4], expected input[5, 3, 32, 32] to have 1 channels, but got 3 channels instead

### Sur Auto_VI

In [107]:
colorization_model_w = ColorizationModel(latent_dim=128)  # Instantiate the colorization model

# Pré-entrainement pour "engine wiring" (w)
colorization_encoder_w = train_ssl_model(
    colorization_model_w,
    dataloader_train_w,
    dataloader_test_w,
    criterion=nn.MSELoss(),
    optimizer=optim.Adam(colorization_model_w.parameters(), lr=0.001)
)

Epoch 1/5, Train Loss: 0.0251, Avg Val Loss: 0.0196
Epoch 2/5, Train Loss: 0.0172, Avg Val Loss: 0.0172
Epoch 3/5, Train Loss: 0.0143, Avg Val Loss: 0.0157
Epoch 4/5, Train Loss: 0.0127, Avg Val Loss: 0.0149
Epoch 5/5, Train Loss: 0.0118, Avg Val Loss: 0.0142


Interprétations.

## Pretext tasks 2 : inpainting model

### Sur MVTec_AD

### Sur Auto_VI

## Pretext tasks 3 : masked autoencoder

### Sur MVTec_AD

### Sur Auto_VI

# II - Partie 2 :
Use the model loss as an anomaly score and evaluate the anomaly score’s
discriminative power by plotting the ROC curve and the AUROC metric.